https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a
https://www.browserstack.com/guide/find-element-by-xpath-in-selenium
https://stackoverflow.com/questions/3030487/is-there-a-way-to-get-the-xpath-in-google-chrome
https://www.guru99.com/selenium-webtable.html
https://stackoverflow.com/questions/19200497/python-selenium-webscraping-nosuchelementexception-not-recognized
https://www.w3schools.com/python/python_lists_add.asp


USe try and catch exceptions to stop iterating and go to the next page or line
https://www.scrapingbee.com/blog/selenium-python/

Selecting from a dropdown
https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python

waiting for page load
https://stackoverflow.com/questions/26566799/wait-until-page-is-loaded-with-selenium-webdriver-for-python
https://www.browserstack.com/guide/selenium-wait-for-page-to-load
https://www.askpython.com/python/examples/python-wait-for-a-specific-time

Possible CLI
https://github.com/markfink/whaaaaat

Adding a database
https://realpython.com/python-mysql/

Could maybe pull the table and then parse the table locally using something like pandas read_html rather than going through cell by cell


1. run loop for the column until the end of the row where NoSuchEl exception is trigger
2. have the outer loop move to the next column
3. run loop for the column until the end of the row where NoSuchEl exception is trigger
4. continue until last column which which will tirgger another no such elem exception
5. have selenium click the next page (third outer loop)
6. repeat steps 1-4 until an error is triggered which stops all looping
7. clean up text and send to mySQL db


Header start and end xpaths, use this for the outer loop
Start: //*[@id="pagedResults"]/div[2]/table/thead/tr/th[1]
End: //*[@id="pagedResults"]/div[2]/table/thead/tr/th[17]

Row 1, col 1
//*[@id="pagedResults"]/div[2]/table/tbody/tr[1]/td[1]

Ro1 1, col 2
//*[@id="pagedResults"]/div[2]/table/tbody/tr[1]/td[2]

In [13]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

from getpass import getpass
from mysql.connector import connect, Error

In [14]:
#driver = webdriver.Chrome(r"C:/Users/Charlie/Documents/GitHub/chromedriver.exe")
driver = webdriver.Chrome(r"C:/Users/user/Documents/GitHub/chromedriver.exe")

In [15]:
def get_data_sve (url, event):
    driver.get(url)  #pull up the page
    time.sleep(5)  #wait for the page to load

    #Select the data set
    catagory = Select(driver.find_element_by_id('SelectedDivisionName'))  #go to the drop down
    catagory.select_by_visible_text(event) #select from the dropdown
    apply = driver.find_element_by_xpath('//*[@id="searchEventResults"]/div/div/span/button')  #select the apply button
    apply.click()  #click the apply button
    time.sleep(2)  #wait for the page to load

    #determine how many pages for each even need to be scrolled through
    page_num_text = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/div/div/span[2]').text  # select the element with the page numbers and get its text
    page_num_list = page_num_text.split(" ")  #split the string up
    page_num = int(page_num_list[1])  #get the number
    #print(page_num) #print it for validation

    page_num = 2  # overide for debugging
    
    data_vals = dict.fromkeys(
        ['Bib', 'Name', 'Team Name', 'City', 'Start Time', 'Swim', 'T1', 'Bike', 'T2', 'Run', 'Chip Elapsed',
         'Division', 'Div Place', 'Age', 'Age Place', 'Gender', 'Gender Place'])

    #pull the data off the page
    #need to update this so the dictionary gets populated cleanly
    #maybe have the inner loop add the values directly to the dictionary key
    for page in range(1, page_num+1):
        print("Pulling from page: " + str(page)) #let you know what page you are on
        i = 1   #counter for the columns
        for key in data_vals:
            rows = []
            for r in range(1, 55):
                try:    #stop the loop when you run out of data to pull
                    rows.append(driver.find_element_by_xpath(
                        '//*[@id="pagedResults"]/div[2]/table/tbody/tr[' + str(r) + ']/td[' + str(i) + ']').text)
                    # print(rows)
                except  NoSuchElementException:
                    # print('nse')
                    break
            if data_vals[key] is None:  #deal with the empty dict edge case
                data_vals[key] = rows
            else:
                data_vals[key].extend(rows)
            i = i + 1

        # click to the next page of records
        print("Going to next page")
        apply = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/ul/li[3]/a')
        apply.click()
        #wait for results to load
        print("Waiting for page to load")
        time.sleep(5)
        print("Done waiting")

    return data_vals

In [16]:
sprint = get_data_sve('https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results', 'SPRINT-AGE GROUP')

Pulling from page: 1
Going to next page
Waiting for page to load
Done waiting
Pulling from page: 2
Going to next page
Waiting for page to load
Done waiting


In [18]:
olympic = get_data_sve('https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results', 'OLYMPIC-AGE GROUP')

Pulling from page: 1
Going to next page
Waiting for page to load
Done waiting
Pulling from page: 2
Going to next page
Waiting for page to load
Done waiting


In [19]:
df = pd.DataFrame(sprint)

In [20]:
df.head()

,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
0,1051,MANUEL HUECKEL,TEAM WY55,SAN FRANCISCO,09:35:00,00:10:11,00:03:08,00:31:19,00:00:44,00:19:10,01:04:34,SPRINT-AGE GROUP,1,23,O/A,M,1
1,978,TYLER CURRAN,,WHITE PLAINS,09:35:00,00:09:28,00:02:59,00:31:42,00:00:52,00:20:14,01:05:16,SPRINT-AGE GROUP,2,24,O/A,M,2
2,924,BRIAN BERGER,GOLD COUNTRY TRIATHLON CLUB,LINCOLN,09:35:00,00:09:33,00:03:53,00:33:12,00:01:25,00:22:02,01:10:08,SPRINT-AGE GROUP,3,25,O/A,M,3
3,1187,CLAY ROBBINS,,REDWOOD CITY,09:35:00,00:11:30,00:03:42,00:34:03,00:01:37,00:19:26,01:10:19,SPRINT-AGE GROUP,4,30,1,M,4
4,943,ANDREW BUNNER,,BELMONT,09:40:00,00:11:46,00:03:49,00:33:34,00:01:06,00:20:54,01:11:12,SPRINT-AGE GROUP,5,44,1,M,5


## Can directly pull the table from the web page and send it to a very logn string. This is much faster then reading each cell but needs further implementation work
- Need to have it cycle through each page
- need to set up splits, can likely be done by splitting on "<a class="text-overflow-ellipsis">" and "</"

In [22]:
titles = driver.find_elements_by_css_selector('#pagedResults > div.table-responsive > table > tbody')

In [23]:
#can use split to parse this. will need to loop it once the whole set gets pulled

table = titles[0].get_attribute('innerHTML') #returns all the html in the webelement as a string
print(table)


                <tr class="clickable" data-bib-number="86">
                        <td>
                                    <span class="text-overflow-ellipsis">86</span>
                        </td>
                        <td>
                                <a class="text-overflow-ellipsis">TERRY CANALE</a>
                        </td>
                        <td>
                                    <span class="text-overflow-ellipsis">TEAM ZOOT</span>
                        </td>
                        <td>
                                    <span class="text-overflow-ellipsis">FREMONT</span>
                        </td>
                        <td>
                                    <span class="text-overflow-ellipsis">08:30:00</span>
                        </td>
                        <td>
                                    <span class="text-overflow-ellipsis">00:29:28</span>
                        </td>
                        <td>
                                  

In [27]:
split_string_1 = str(table)
split1 = split_string_1.split(r'">')

In [25]:
#need either lambda function or list comprehension

split2 = []
for entry in split1:
    split2.extend(entry.split('</'))

In [26]:
engine = create_engine('sqlite:///data.db', echo=False)

In [28]:
df.to_sql('data.db', con=engine, index_label='ID')

In [ ]:
#this is a test comment